# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [6]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [7]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [8]:
# Combine all features
X_train = X_train_numeric.join(X_train_cat)
X_val = X_val_numeric.join(X_val_cat)
X_test = X_test_numeric.join(X_test_cat)

As a refresher, preview the training data: 

In [9]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [10]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [11]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()
baseline_model.add(layers.Input((296, )))

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation="relu"))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation="relu"))

# Output layer
baseline_model.add(layers.Dense(1, activation="linear"))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [12]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()
ss_X.set_output(transform="pandas")

# Fit and transform train data
X_train_scaled = ss_X.fit_transform(X_train[numeric_columns])

# Transform validate and test data
X_val_scaled = ss_X.transform(X_val[numeric_columns])
X_test_scaled = ss_X.transform(X_test[numeric_columns])

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [13]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [14]:
# Train the model
normalized_input_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val)
)

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [16]:
# Instantiate StandardScaler
ss_y = StandardScaler()
ss_y.set_output(transform="pandas")

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [17]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
results = normalized_model.fit(
    X_train, y_train_scaled,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.7116 - mse: 0.7116 - val_loss: 0.1953 - val_mse: 0.1953
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2702 - mse: 0.2702 - val_loss: 0.1461 - val_mse: 0.1461
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1902 - mse: 0.1902 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2069 - mse: 0.2069 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1852 - mse: 0.1852 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1272 - mse: 0.1272 - val_loss: 0.1212 - val_mse: 0.1212
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1353 - mse: 0.1353 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1044 - mse: 0.1044 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.10

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
results.history["mse"][-1]

0.010375057347118855

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [19]:
# Evaluate the model on validate data
results.history["val_mse"][-1]

0.13335992395877838

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [20]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [21]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Input(n_features))
he_model.add(layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(
    optimizer="SGD",
    loss="mse",
    metrics=["mse"]
)

# Train the model
results_he = he_model.fit(
    X_train, y_train_scaled,
    batch_size=32,
    epochs=150,
    validation_data=(X_val, y_val_scaled)
)

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.7209 - mse: 0.7209 - val_loss: 0.2395 - val_mse: 0.2395
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2401 - mse: 0.2401 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2367 - mse: 0.2367 - val_loss: 0.1812 - val_mse: 0.1812
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2151 - mse: 0.2151 - val_loss: 0.1720 - val_mse: 0.1720
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1404 - mse: 0.1404 - val_loss: 0.1573 - val_mse: 0.1573
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1388 - mse: 0.1388 - val_loss: 0.2122 - val_mse: 0.2122
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1900 - mse: 0.1900 - val_loss: 0.1540 - val_mse: 0.1540
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0923 - mse: 0.0923 - val_loss: 0.1577 - val_mse: 0.1577
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.11

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
results_he.history["mse"][-1]


0.008678365498781204

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [28]:
# Evaluate the model on validate data
results_he.history["val_mse"][-1]


0.16012044250965118

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [29]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Input(n_features))
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["mse"]
)

# Train the model
results_lecun = lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.7493 - mse: 0.7493 - val_loss: 0.1844 - val_mse: 0.1844
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2750 - mse: 0.2750 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2153 - mse: 0.2153 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1457 - mse: 0.1457 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1282 - mse: 0.1282 - val_loss: 0.1404 - val_mse: 0.1404
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1258 - mse: 0.1258 - val_loss: 0.1269 - val_mse: 0.1269
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1150 - mse: 0.1150 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0949 - mse: 0.0949 - val_loss: 0.1241 - val_mse: 0.1241
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.10

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
results_lecun.history["mse"][-1]

0.008971946313977242

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [31]:
# Evaluate the model on validate data
results_lecun.history["val_mse"][-1]

0.17557209730148315

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [26]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(
    optimizer="rmsprop",
    loss="mse",
    metrics=["mse"]
)

# Train the model
results_rms = rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.4455 - mse: 0.4455 - val_loss: 0.1421 - val_mse: 0.1421
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1337 - mse: 0.1337 - val_loss: 0.1190 - val_mse: 0.1190
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1254 - mse: 0.1254 - val_loss: 0.1517 - val_mse: 0.1517
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0898 - mse: 0.0898 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0883 - mse: 0.0883 - val_loss: 0.1963 - val_mse: 0.1963
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0674 - mse: 0.0674 - val_loss: 0.1802 - val_mse: 0.1802
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0708 - mse: 0.0708 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0461 - mse: 0.0461 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.03

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
results_rms.history["mse"][-1]

0.005162755027413368

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [33]:
# Evaluate the model on validate data
results_rms.history["val_mse"][-1]

0.08847854286432266

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [35]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mse"]
)

# Train the model
results_adam = adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.6341 - mse: 0.6341 - val_loss: 0.1532 - val_mse: 0.1532
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1879 - mse: 0.1879 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0861 - mse: 0.0861 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0714 - mse: 0.0714 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0649 - mse: 0.0649 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0441 - mse: 0.0441 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0453 - mse: 0.0453 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0282 - mse: 0.0282 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.02

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [36]:
# Evaluate the model on training data
results_adam.history["mse"][-1]


0.0006410202477127314

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [37]:
# Evaluate the model on validate data
results_adam.history["val_mse"][-1]


0.1092066764831543

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [38]:
# Evaluate the best model on test data
predictions = rmsprop_model.predict(X_test)
mean_squared_error(y_test_scaled, predictions)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


0.14351577799539714

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [39]:
# Generate predictions on test data
y_test_pred_scaled = predictions

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test_pred, predictions))

197909.23

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.